In [1]:
import ee
ee.Authenticate()
ee.Initialize()
import geemap
ee.Initialize()
import time
import sys
sys.path.append(r"C:\Users\JoaoPereira\Desktop\pygeo\Geospatial-Ops")
import aux_functions as af
import importlib
importlib.reload(af)

<module 'aux_functions' from 'c:\\Users\\JoaoPereira\\Desktop\\pygeo\\Geospatial-Ops\\aux_functions.py'>

# Search by WKT

In [2]:
# Your rectangle WKT string, use https://geojson.io/ and save as WKT (open in a text editor to copy the coordinates)

#WKT
wkt_string = "POLYGON ((-9.549958547993015 42.28532298469514, -9.644998817563106 39.54941956811506, -9.549958547993015 36.86237642690354, -6.096914983741101 36.86237642690354, -6.096914983741101 42.28532298469514, -9.549958547993015 42.28532298469514))"

# Manual conversion from WKT to a list of coordinates for Earth Engine
coordinates_str = wkt_string.strip('POLYGON ((').strip('))').split(', ')
coordinates = [[float(coord) for coord in point.split(' ')] for point in coordinates_str]

ee_geometry = ee.Geometry.Polygon([coordinates])

location = ee_geometry.coordinates().getInfo()[::-1]
location

[[[-9.549958547993015, 42.28532298469514],
  [-9.644998817563106, 39.54941956811506],
  [-9.549958547993015, 36.86237642690354],
  [-6.096914983741101, 36.86237642690354],
  [-6.096914983741101, 42.28532298469514],
  [-9.549958547993015, 42.28532298469514]]]

# Dataset Load

In [3]:
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic()

# Mask values below 0 meters, not creating a binary mask
dem_land = dem.updateMask(dem.gte(0))

band_names = dem_land.bandNames().getInfo()
print('Band names:', band_names)

Band names: ['b1']


In [4]:
# Assuming there's only one band, typically this is the case with DEMs
band_name = band_names[0]

# Find minimum and maximum values
stats = dem_land.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee_geometry,
    scale = 30,
    maxPixels=1e13
)

# Print the stats dictionary to inspect the keys
print(stats.getInfo())

# Fetch the results using the correct key names
min_val_key = f'{band_name}_min'
max_val_key = f'{band_name}_max'

min_val = stats.getInfo()[min_val_key]
max_val = stats.getInfo()[max_val_key]

print('Min value:', min_val)
print('Max value:', max_val)

{'b1_max': 2116.7900390625, 'b1_min': 0}
Min value: 0
Max value: 2116.7900390625


# Import Geemap for visualization

In [5]:
Map = geemap.Map()

# Add Google Satellite as the basemap
Map.add_basemap('SATELLITE')

Map.addLayer(dem_land, {
    'min': min_val,
    'max': max_val,
    'palette': ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
}, 'Elevation (meters)')

# Add the WKT outline as a separate layer
Map.addLayer(ee_geometry, {'color': 'red'}, 'AOI Outline')

Map.centerObject(ee_geometry, 7)  # This represents the zoom level
Map

Map(center=[39.551584683648464, -7.847325591035253], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
# Set the description variable to be used as the tiff name
description = 'PT_fabdem_3857'

In [7]:
# Set up export parameters
task = ee.batch.Export.image.toDrive(**{
    'image': dem_land.clip(ee_geometry),
    'description': description,
    'scale': 30, # Set the spatial resolution
    'region': ee_geometry,
    'folder': 'FABDEM_DB',
    'fileFormat': 'GeoTIFF',
    'crs': 'EPSG:3857',
    'maxPixels': 1e13
})

# Start the export task
task.start()

# Optional: Check the task status
print('Export task started with task ID:', task.id)

# Monitor the task until completion
while task.active():
    print('Task still running...')
    time.sleep(60)  

print('Task completed')

Export task started with task ID: RUYPTVXILEYBPLMV7QPTDE7M
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task completed
